In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time 
import csv

Import Data

In [ ]:
all_data= pd.read_excel('output.xlsx')
all_data['Date + Time'] = pd.to_datetime(all_data['Date + Time'])
all_data['Date'] = pd.to_datetime(all_data['Date + Time']).dt.date

all_data['Same Bound'] = all_data['Bound'] == all_data['Bound'].shift(1)
all_data['Same Lane'] = all_data['Lane'] == all_data['Lane'].shift(1)    

all_data.loc[(all_data['Same Bound'] == True) & (all_data['Same Lane'] == True), 'Gap Time'] = all_data['Date + Time'].diff().dt.seconds
all_data.loc[all_data['Same Bound'] == False, 'Gap Time'] = pd.NA
all_data.loc[all_data['Same Lane'] == False, 'Gap Time'] = pd.NA

cols = ['Gap Time']
all_data.loc[:,cols] = all_data.loc[:,cols].bfill()



Renaming of Unamed Columns

In [ ]:
for i in range(11,27):
    num = (i - 7) // 2
    if i % 2 == 0:
        new_column_name = 'Axle Spacing ' + str(num)
    else:
        new_column_name = 'Axle Weight ' + str(num)
    column_name = 'Unnamed: ' + str(i)
    print(f'{column_name=}, {new_column_name=}')
    all_data.rename(columns={column_name: new_column_name}, inplace=True)
all_data.columns


## Imported From 'Data_handle.ipynb'

In [ ]:
from datetime import time as t

def vehicular_flow_in_hours_setup():
        
    def generate_time(time_list):
        for h in range(24):
            time_list.append(t(h).strftime("%H"))
        return

    # Create the Time column, where it is in the string format of hour:minute in 1 minute increments
    all_data['Time'] = all_data['Date + Time'].dt.floor('h')
    all_data['Time'] = all_data['Time'].dt.strftime("%H")

    # Make a time list to store all the x data from 00:00 to 23:59
    time_list = []
    generate_time(time_list)
    dict = {"Time": time_list}

    # Loops over the number of recorded date
    date_list = all_data['Date'].unique()
    for date in date_list:
        vehicle_flow = []
        # Loops over each minute
        for time in time_list:
            # Tallies the number of vehicle that is recorded in that minute. -> convert the unit from veh/min to veh/hr
            # ? Times 60 for the total numebr of cars to convert: cars per min to cars per hour
            tmp = all_data[(all_data['Date']==date) & (all_data['Time']==time)]
            hourly_flow = len(tmp.index)
            if hourly_flow < 100: 
                hourly_flow = 0
            vehicle_flow.append(hourly_flow)
        dict[date] = vehicle_flow
    # print()
    VF_testing = pd.DataFrame(dict)

    # Finding the Average Values
    VF_testing['Total'] = VF_testing.iloc[:,1:6].sum(axis=1)
    VF_testing['Unique Dates'] = VF_testing.iloc[:,1:6].astype(bool).sum(axis=1)

    VF_testing['Average'] = VF_testing['Total'] / VF_testing['Unique Dates']
    VF_testing['Average'] =  VF_testing['Average'].fillna(0)
    
    return VF_testing, time_list, date_list

def average_vehicular_flow():
    VF_testing, time_list, date_list = vehicular_flow_in_hours_setup()
    np_vehicular_flow = VF_testing['Average'].to_numpy() 
    np_time_list = time_list
    print(f'{np_vehicular_flow=}')
    print(f'{np_time_list=}')
    return None



def vehicular_flow_in_hours_with_average_and_scatter():

    VF_testing, time_list, date_list = vehicular_flow_in_hours_setup()

    # graph plot the average value
    plt.plot(time_list, VF_testing['Average'],'.-', label='Average')
    print(VF_testing['Average'])

    # Scatter Plot
    for date in date_list:
        # removes the time data for when there is no vehicle recorded
        tmp = np.array(VF_testing[date].astype(float))
        tmp[tmp==0] = np.nan
        
        plt.scatter(VF_testing['Time'], tmp, label=date)

    plt.xticks(time_list, rotation=70)
    plt.legend(loc=0)
    plt.grid()
    plt.xlabel('Time of Day (24 hour format)')
    plt.ylabel('Vehicular Flow (veh/h)')
    plt.title('Hourly Vehicular Flow per Lane Across Different Times of Day')
    plt.tight_layout()
    plt.show()
    return 
vehicular_flow_in_hours_with_average_and_scatter()

# Setup Function 

### For generate_daily_vehicle_stream()

In [ ]:
# * Function Template for finding distribution from dataset, plotting the dataset, and interpolating data
from importlib import reload
import random
from collections import Counter

database = {}

def filter_all_data(vehicle_class_column_name, vehicle_class_value, df=all_data):
    return df[df[vehicle_class_column_name] == vehicle_class_value]

# optimised by ChatGPT
def obtain_distribution(total_weight_column_name: str, df: pd.DataFrame = all_data):
    counts = df.groupby(total_weight_column_name).size().reset_index(name='Count')
    total_rows = counts['Count'].sum()
    counts['Percentage'] = counts['Count'] / total_rows
    # counts['Cumulative Probability'] = (counts['Count'] / total_rows).cumsum()

    counts['Cumulative Probability'] = counts['Percentage'].cumsum()
    # * Numpy
    # np_cumu_distribution = counts[[total_weight_column_name, 'Cumulative Probability']].to_numpy().T
    # return np_cumu_distribution

    # * Pandas
    # output = counts.loc[:,[total_weight_column_name,'Cumulative Probability' ]]    
    output = counts.loc[:,[total_weight_column_name,'Percentage' ]]    
    return output

    




def add_unit(axis: str):
    if axis == 'Gap Time': 
        axis += ' (s)'
    elif axis == 'Total Weight': 
        axis += ' (kg)'
    elif axis == 'Speed': 
        axis += ' (km/h)'
    elif 'Axle Weight' in axis: 
        axis += ' (kg)'
    elif 'Axle Spacing' in axis: 
        axis += ' (mm)'
    return axis

def plot_distribution(df:pd.DataFrame, additional_string:str = ''):
    import matplotlib.pyplot as plt 

    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot()

    ax.set_title(f'For Cars of {additional_string}:',fontweight='bold')
    x_axis = df.columns[0]
    y_axis = df.columns[-1]

    tmp_x_axis = df.columns[0]

    tmp_y_axis = df.columns[-1]

    if (x_axis == 'Bound')| (x_axis == 'Date + Time')| (x_axis == 'Seq No')| (x_axis == 'Lane')| (x_axis == 'Class')| (x_axis == 'No of Axle'): 
        ax.bar(df[x_axis], df[y_axis])
    else:
        # ax.scatter(df[x_axis], df[y_axis], s=20)
        ax.plot(df[x_axis], df[y_axis],color='C0', marker='o', linestyle='-' )

    ax=reload(plt)
    if y_axis == 'Cumulative Probability':
        fig.suptitle(f'CDF of {x_axis}')
    else:
        fig.suptitle(f'{x_axis} against {y_axis}')

    # print(f'{df.iloc[-1,0]}=')
    ax.xlim(0,df.iloc[-1,0])
    # Show the major grid and style it slightly.
    plt.grid(True, which='both')
    plt.grid(which='major', color='black', linewidth=0.8)
    # Show the minor grid as well. Style it in very light gray as a thin,
    # dotted line.
    plt.grid(which='minor', color='black', linestyle=':', linewidth=1)
    # Make the minor ticks and gridlines show.
    plt.minorticks_on()

    plt.legend(loc=0)
    
    x_axis = add_unit(x_axis)
    y_axis = add_unit(y_axis)

    ax.xlabel(x_axis)
    ax.ylabel(y_axis)

    # from pathlib import Path

    filename = f'Graphs/CPD_{additional_string}_{tmp_y_axis}_{tmp_x_axis}.png'
    # my_file = Path(filename)
    # if not my_file.is_file():
    # file exists
    plt.savefig(filename)
   
    return

# Optimised by ChatGPT
def generate_and_interpolate_value(numpy_array: np.float64, total_weight_key: str):

    numpy_array = numpy_array.to_numpy().T


    random_prob = random.random()
    if total_weight_key in ['Bound', 'Date + Time', 'Seq No', 'Lane', 'Class', 'No of Axle']:
        sorter = np.argsort(numpy_array[1])
        tmp = np.searchsorted(numpy_array[1],[random_prob,],side='left', sorter=sorter)[0]

        interpolated_value = int(numpy_array[0][tmp])

    else:
        interpolated_value = np.interp(random_prob, numpy_array[1], numpy_array[0])


    return interpolated_value


def double_filter_and_generate_variable(vehicle_class_key1: str, vehicle_class_value1: int, vehicle_class_key2: str, vehicle_class_value2: int , total_weight_key: str, )-> float :
    
    # * Timesaving techniques!
    # Find the set of queries from the database to see if it is run before or not
    filter_key = f'{str(vehicle_class_key1)} = {str(vehicle_class_value1)} and {str(vehicle_class_key2)} = {str(vehicle_class_value2)}'
    database_key = f'{filter_key} {str(total_weight_key)}'
    # print(f'DATABASE KEY IS {database_key}')

    #* If it has been run before 

    if database_key in database:
        # * call the distribution from database
        distribution = database[database_key]
        # print(f'RECALLED DISTRIBUTION FROM DATABASE WITH KEY {database_key}')

    # * If it has never been run before
    else:
        # * generate the distribution from scratch
        # * filtering the data twice 
        tmp_df = filter_all_data(vehicle_class_key1, vehicle_class_value1)
        tmp_df = filter_all_data(vehicle_class_key2, vehicle_class_value2,df=tmp_df)

        # * Now we count the data that is within the filtered dataframe to create CDF
        distribution = obtain_distribution(total_weight_key, tmp_df)
        
        # * Update the database so it has the new query 
        # print(f'ASSIGNED DISTRIBUTION TO DATABASE WITH KEY {database_key}')
        database[database_key] = distribution
    
    # * Return Error when distribution found is empty. 
    # if distribution.empty:
    #     print(f'Dtaframe with key = {total_weight_key} is empty.')
    #     return

    #* Plots the distribution
    # plot_distribution(distribution, filter_key)

    #* Generates a random num (0,1), then finds its corresponding value
    output = generate_and_interpolate_value(distribution, total_weight_key)

    return output



def generate_variable(vehicle_class_key: str, vehicle_class_value: int , total_weight_key: str)-> float :
    
    # * Timesaving techniques!
    # Find the set of queries from the database to see if it is run before or not
    filter_key = f'{str(vehicle_class_key)} = {str(vehicle_class_value)}'
    database_key = f'{filter_key} {str(total_weight_key)}'


    #* If it has been run before 
    if database_key in database:
        # * call the distribution from database
        distribution = database[database_key]
        # print(f'RECALLED DISTRIBUTION FROM DATABASE WITH KEY {database_key}')
    
    # * If it has never been run before
    else:
        # * generate the distribution from scratch
        # * filtering the data once

        # start = time.time()
        tmp_df = filter_all_data(vehicle_class_key, vehicle_class_value)
        # end = time.time()
        # print(f'time for filter_all_data is {end-start}')
        
        # * Now we count the data that is within the filtered dataframe to create CDF
        # start = time.time()
        distribution = obtain_distribution(total_weight_key, tmp_df)
        # end = time.time()
        # print(f'time for obtain_distribution is {end-start}')
        
        # * Update the database so it has the new query 
        database[database_key] = distribution
    
    # * Return Error when distribution found is empty. 
    # if len(distribution) == 0:
    #     print(f'Dtaframe with key = {total_weight_key} is empty.')
    #     return
    
    #* Plots the distribution
    plot_distribution(distribution, filter_key)

    #* Generates a random num (0,1), then finds its corresponding value
    # start = time.time()
    output = generate_and_interpolate_value(distribution, total_weight_key)
    # end = time.time()
    # print(f'time for generate_and_interpolate_value is {end-start}')

    return output


def normalise_axle_weights(axle_weight_list: list):
    sum = 0
    for axle_weight in axle_weight_list:
        sum += axle_weight
    output = np.array([])
    for axle_weight in axle_weight_list:
        output = np.append(output, axle_weight / sum)
    return output

def distribute_total_weight(axle_weight_list: list, GVW: float):
    output = np.array([])
    for axle_weight_proportion in axle_weight_list:
        output = np.append(output, axle_weight_proportion * GVW)
    return output

    



# Minor Functions

### For generate_daily_vehicle_stream()

In [ ]:
import random

def generate_vehicle_class():
    
    # Hard Code
    cumu_proportion = np.array([0.3712258 , 0.70580101, 1.        ])
    cumu_distribution = np.array([0.00000000e+00, 6.65190000e-04, 5.33101100e-02, 1.25910680e-01,
       2.88438400e-01, 3.74976250e-01, 5.11910050e-01, 8.76781760e-01,
       1.00000001e+00])

    
    roll = random.uniform(0, 1)
    if roll < cumu_proportion[0]: # HDV
        # * determine detailed class 3-9
        # distribution:list[float] = finding_HDV_class_distribution()
        # cumu_distribution = np.array([0.00000000e+00, 6.65190000e-04, 5.33101100e-02, 1.25910680e-01,2.88438400e-01, 3.74976250e-01, 5.11910050e-01, 8.76781760e-01,1.00000001e+00])
        output = np.searchsorted(cumu_distribution,[random.uniform(0, 1),],side='left')[0]
        return 1, output
    elif roll < cumu_proportion[1]: # Private Car
        return 2, 2
    else: # Motorbike
        return 3, 1

def generate_GVW(vehicle_class: int, EV_proportion: float) -> float:
    output = generate_variable('Class', vehicle_class, 'Total Weight')
    return (output * 1.3 if random.uniform(0,1) <= EV_proportion else output )
        

def generate_axle_number(vehicle_class: int) -> int:
    return  generate_variable('Class', vehicle_class, 'No of Axle')

def generate_axle_spacing(vehicle_class: int, axle_number: int) -> list[float]:
    # finds the axle spacing based on the vehicle_class + axle_number
    # find the subset of data that matches with the right vehicle class, then match with the right axle number
    # find the cumulative probabiility distribution of each/all axle spacing
    output = np.array([])
    for spacing_index in range(2,axle_number+1):
        axle_spacing_column_name =  'Axle Spacing '+ str(spacing_index)
        output = np.append(output, double_filter_and_generate_variable('Class', vehicle_class,'No of Axle',axle_number,axle_spacing_column_name))
    return output


def generate_axle_weight(vehicle_class: int, axle_number: int, GVW: float)-> list[float]:
    output = np.array([])
    for spacing_index in range(1,axle_number+1):
        axle_weight_column_name =  'Axle Weight '+ str(spacing_index)
        output = np.append(output,double_filter_and_generate_variable('Class', vehicle_class,'No of Axle',axle_number,axle_weight_column_name))
    
    # * normalise and distribute GVW to axle weights. 
    output = normalise_axle_weights(output)
    output = distribute_total_weight(output, GVW)
    return output

def generate_speed(lane_num: int)-> float:
    return generate_variable('Lane', lane_num, 'Speed')

def generate_gap_time(vehicle_class: int)-> float:
    output = generate_variable('Class', vehicle_class, 'Gap Time')
    if type(output) == pd.Series : 
        output = float(output.iloc[0])
    return output

def determine_hourly_flow_rate(hour: int)-> float:
    # obtain target flow rate based on the hourly flow rate
    np_avg_veh_flow = np.array([1746.        , 1155.        ,  760.        ,    0.        ,
        0.        ,    0.        ,    0.        ,    0.        ,
    4709.        , 5161.        , 4768.        , 3046.        ,
    3073.5       , 3482.        , 3177.        , 2696.        ,
    2340.        , 2299.5       , 2262.        , 1763.5       ,
    1146.        , 3839.66666667, 1582.        ,  658.        ],dtype='i')
    return np_avg_veh_flow[hour]



### For load_assessment()


In [ ]:
def count_cars_on_bridge_at_instance(np_generated_traffic_for_the_day: np.float64| np.int8, timer: int, bridge_length: int, index_of_first_veh_on_bridge_last_second: int, index_of_last_veh_on_bridge_last_second: int):
    
    # print(f'AT THE START {np_generated_traffic_for_the_day[-1][:10]=}')
    # * Find the last car on the bridge
    index_of_last_veh_on_bridge = index_of_last_veh_on_bridge_last_second + 1
    while True:
        
        if index_of_last_veh_on_bridge >= len(np_generated_traffic_for_the_day[0])-1:
            break


        if np_generated_traffic_for_the_day[5][index_of_last_veh_on_bridge] > timer:
            index_of_last_veh_on_bridge -= 1
            break
        else: 
            index_of_last_veh_on_bridge += 1
    
            
    
    index_of_first_veh_on_bridge = index_of_last_veh_on_bridge + 1
    for car_index in range(index_of_first_veh_on_bridge_last_second,index_of_last_veh_on_bridge+1):
        distance_from_start = (timer - np_generated_traffic_for_the_day[5][car_index]) * (np_generated_traffic_for_the_day[4][car_index]/ 3.6)



        if distance_from_start <= bridge_length:
            index_of_first_veh_on_bridge = car_index 
            break
    else:
        index_of_first_veh_on_bridge = index_of_last_veh_on_bridge+1

    if index_of_first_veh_on_bridge > index_of_last_veh_on_bridge: 
        return np.array([]), index_of_first_veh_on_bridge, index_of_last_veh_on_bridge
    else:
        
        np_cars_on_bridge = np_generated_traffic_for_the_day.T[index_of_first_veh_on_bridge:index_of_last_veh_on_bridge+1].T

        return np_cars_on_bridge ,index_of_first_veh_on_bridge, index_of_last_veh_on_bridge

def setup_list_for_next_HDV_time(np_generated_daily_traffic: np.float64| np.int8, np_generated_traffic_for_the_day: np.float64| np.int8):
    next_HDV_time_list = np.array([])
    for index in range(len(np_generated_daily_traffic)):
        if (np_generated_daily_traffic[index][1] == float(1)): 
            next_HDV_time_list = np.append(next_HDV_time_list, np.array(index))
    next_HDV_time_list = np.take(np_generated_traffic_for_the_day[5], next_HDV_time_list.astype(int))
    return next_HDV_time_list.astype(int)


def advance_timer_to_next_HDV(np_generated_traffic_for_the_day: np.float64| np.int8, vehicle_indexing_from_last_call: dict[int], total_lane: int, timer: int)-> int:
    
    next_HDV_cumu_gap_time = 1000000
    for index in range(len(np_generated_traffic_for_the_day[0])):

        if (np_generated_traffic_for_the_day[1][index] == float(1)) & (np_generated_traffic_for_the_day[5][index] > timer ) & (  np_generated_traffic_for_the_day[5][index] < next_HDV_cumu_gap_time): 
            next_HDV_cumu_gap_time = np_generated_traffic_for_the_day[5][index]

    


        
        
    return next_HDV_cumu_gap_time


def convert_series_type(*items) -> (tuple | float):
    if len(items) == 1:
        if type(items) == pd.Series:
            items = float(items.iloc[0])
        return items
    for index, item in enumerate(items):
        if type(item) == pd.Series:
            items[index] = item.iloc[0]
    return items

def calculate_BM(car_distance_from_start: float ,axle_weight: float , total_axle_spacing: float, bridge_length: int): 
    


    
    
    
    reaction_force_at_support = axle_weight * (bridge_length - car_distance_from_start) / (bridge_length)
    
    
    moment_arm = car_distance_from_start
    
    
    BM = reaction_force_at_support * car_distance_from_start

    BM =  axle_weight * (bridge_length - car_distance_from_start) / (bridge_length) / car_distance_from_start
    
    return BM


def calculate_mid_span_BM(np_cars_on_bridge: np.float64|np.int8, bridge_length: int, top_3_BM_recorded: list[float], timer: int, top_3_BM_one_lane_recorded: np.int16, number_of_measured_lanes: str): 

    
    max_possible_BM = np.sum(np_cars_on_bridge[3]) * bridge_length / 4


    
    if (number_of_measured_lanes == 'one lane') & ((max_possible_BM) <= min(top_3_BM_one_lane_recorded)): 
        return 0 , np.array([])
    elif (number_of_measured_lanes == 'all lane') & ((max_possible_BM) <= min(top_3_BM_recorded)):
        return 0 , np.array([])
    total_BM = 0
    transposed_np_cars_on_bridge = np_cars_on_bridge.T
    output = np.array([])

    for element in transposed_np_cars_on_bridge:

        car = element

        if car[1] == float(2) or car[1] == float(3):
            continue

    


    
        car_distance_from_start = int((timer - car[5]) * (car[4] / 3.6))


        if ((car_distance_from_start > 0) & (car_distance_from_start < 20)):
            if (number_of_measured_lanes == 'all lane'):
                axle_number = generate_axle_number(car[2])
                if len(output) == 0: 
                    output = np.append(car,axle_number)
                else:
                    output = np.vstack((output, np.append(car,axle_number)))


            
            total_BM += car[3] * car_distance_from_start / 2
    if (number_of_measured_lanes == 'all lane'):
        output = output.T
    return total_BM, output

# Major Functions

In [ ]:
def generate_a_vehicle(np_generated_traffic: np.float64|np.int8, lane_num: int, timer: float, hour: int, cumu_gap_time: float, EV_proportion: float):
    
    vehicle_type, vehicle_class = generate_vehicle_class()
    speed = generate_speed(lane_num)
    gap_time = generate_gap_time(vehicle_class)

    if vehicle_type == 1:
        GVW = generate_GVW(vehicle_class, EV_proportion)
    else: 
        GVW = 0



    if len(np_generated_traffic) == 0:
        cumu_gap_time = 0
    else: 
        cumu_gap_time += gap_time
    timer = cumu_gap_time 



    np_new_vehicle = np.array([lane_num,vehicle_type,vehicle_class,GVW,speed,cumu_gap_time],dtype=float)

    if len(np_generated_traffic) == 0:
        np_generated_traffic = np_new_vehicle
    else: 
        np_generated_traffic = np.vstack((np_generated_traffic, np_new_vehicle))
        
    

    return np_generated_traffic, timer, cumu_gap_time


def generate_hourly_vehicle_stream(np_generated_daily_traffic: np.float64|np.int8, hour: int, lane_num: int, timer: float, cumu_gap_time: float, EV_proportion: float, traffic_growth: float, vehicles_per_hour: int):
    np_generated_hourly_traffic = np.array([])
    cumu_gap_time = 0


    for _ in range(int(vehicles_per_hour * traffic_growth)):
        np_generated_hourly_traffic, timer , cumu_gap_time = generate_a_vehicle(np_generated_hourly_traffic, lane_num, timer, hour, cumu_gap_time, EV_proportion)


    total_gap_time = float(np_generated_hourly_traffic[-1][5])
    for index in range(len(np_generated_hourly_traffic)):
        np_generated_hourly_traffic[index][5] = np_generated_hourly_traffic[index][5] * 3600 / total_gap_time + hour * 3600
    


    if len(np_generated_daily_traffic) == 0:
        np_generated_daily_traffic = np_generated_hourly_traffic
    else:
        np_generated_daily_traffic = np.concatenate((np_generated_daily_traffic, np_generated_hourly_traffic),axis=0)
    return  np_generated_daily_traffic


    np_generated_daily_traffic_all_lane = np.array([])

    for lane_num in range(1,total_lane+1):
        timer = 0.0
        cumu_gap_time = 0
        np_avg_veh_flow = np.array([1746.        , 1155.        ,  760.        ,    0.        ,
        0.        ,    0.        ,    0.        ,    0.        ,
        4709.        , 5161.        , 4768.        , 3046.        ,
        3073.5       , 3482.        , 3177.        , 2696.        ,
        2340.        , 2299.5       , 2262.        , 1763.5       ,
        1146.        , 3839.66666667, 1582.        ,  658.        ],dtype='i')

        for hour in range(hours_in_a_day):

            if np_avg_veh_flow[hour] != 0:
                np_generated_daily_traffic_all_lane = generate_hourly_vehicle_stream(np_generated_daily_traffic_all_lane, hour, lane_num, timer, cumu_gap_time, EV_proportion, traffic_growth, np_avg_veh_flow[hour])
        
        
            
    return np_generated_daily_traffic_all_lane


def load_assessment(np_generated_daily_traffic: np.float64|np.int8, top_3_BM_recorded: list[float], top_3_BM_dict_cars_on_bridge: dict, hours_in_a_day: int, total_lane: int, top_3_BM_one_lane_recorded: np.int16 , new_BM_recorded: np.float64, new_BM_dict_cars_on_bridge: dict, new_BM_one_lane_recorded: np.float64, multiprocessing: bool)-> tuple[float, pd.DataFrame]: 
    bridge_length = 50
    seconds_in_a_day = 60 * 60 * hours_in_a_day
    timer = 0 
    vehicle_indexing_from_last_call = {}
    max_cumu_gap_time = 9000000
    previous_next_HDV_time_list_index = 0
    np_generated_traffic_for_the_day = np_generated_daily_traffic.T

    next_HDV_time_list = setup_list_for_next_HDV_time(np_generated_daily_traffic, np_generated_traffic_for_the_day)

    for lane in range(1,total_lane+1):
        vehicle_indexing_from_last_call['first ' + str(lane)] = 0
        vehicle_indexing_from_last_call['last ' + str(lane)] = 0
    max_cumu_gap_time = np_generated_traffic_for_the_day[5][-1]


    np_generated_traffic_in_each_lane = {}
    for lane in range(1,total_lane+1):
        np_generated_traffic_in_each_lane[lane] = np_generated_daily_traffic[np_generated_traffic_for_the_day[0] == lane].T
    
    while timer <= seconds_in_a_day:

        if timer >= max_cumu_gap_time:
            break

        np_cars_on_bridge = np.array([])
        total_load_effect = 0

        for lane in range(1,total_lane+1):
            
            
            np_cars_on_bridge_one_lane , vehicle_indexing_from_last_call['first ' + str(lane)], vehicle_indexing_from_last_call['last ' + str(lane)] = count_cars_on_bridge_at_instance(np_generated_traffic_in_each_lane[lane], timer, bridge_length, vehicle_indexing_from_last_call['first ' + str(lane)], vehicle_indexing_from_last_call['last ' + str(lane)])


            if len(np_cars_on_bridge) == 0:
                np_cars_on_bridge = np_cars_on_bridge_one_lane

            elif (len(np_cars_on_bridge) != 0) & (len(np_cars_on_bridge_one_lane) != 0):
                np_cars_on_bridge = np.concatenate((np_cars_on_bridge, np_cars_on_bridge_one_lane),1)

            if len(np_cars_on_bridge_one_lane) != 0:
                load_effect, np_cars_on_bridge_one_lane = calculate_mid_span_BM(np_cars_on_bridge_one_lane, bridge_length, top_3_BM_recorded, timer, top_3_BM_one_lane_recorded, 'one lane')

                if load_effect > min(top_3_BM_one_lane_recorded):
                    if multiprocessing:
                        np.hstack((new_BM_one_lane_recorded,np.array(load_effect)))
                    else:
                        min_index = np.where(top_3_BM_one_lane_recorded == min(top_3_BM_one_lane_recorded))[0][0]
                        top_3_BM_one_lane_recorded[min_index] = load_effect
                
            if len(np_cars_on_bridge) != 0:
                if lane == total_lane:
                    total_load_effect, np_cars_on_bridge = calculate_mid_span_BM(np_cars_on_bridge, bridge_length, top_3_BM_recorded, timer, top_3_BM_one_lane_recorded , 'all lane')

                    if total_load_effect > min(top_3_BM_recorded):
                        if multiprocessing:
                            new_BM_recorded = np.hstack((new_BM_recorded,np.array(total_load_effect)))
                            new_BM_dict_cars_on_bridge[len(new_BM_recorded)-1] = np_cars_on_bridge

                        else:
                            min_index = np.where(top_3_BM_recorded == min(top_3_BM_recorded))[0][0]
                            top_3_BM_recorded[min_index] = total_load_effect
                            top_3_BM_dict_cars_on_bridge[min_index] = np_cars_on_bridge
        timer += 1
        if len(np_cars_on_bridge) == 0: 
            pos_increment = 1
            while previous_next_HDV_time_list_index + pos_increment < len(next_HDV_time_list)-1:
                next_HDV_time = next_HDV_time_list[previous_next_HDV_time_list_index+pos_increment]
                if next_HDV_time > timer:
                    timer = next_HDV_time
                    previous_next_HDV_time_list_index += pos_increment
                    break
                pos_increment += 1
            else:
                if multiprocessing:
                    return new_BM_recorded, new_BM_dict_cars_on_bridge, new_BM_one_lane_recorded
                else:
                    return top_3_BM_recorded, top_3_BM_dict_cars_on_bridge, top_3_BM_one_lane_recorded

    if multiprocessing:
        return new_BM_recorded, new_BM_dict_cars_on_bridge, new_BM_one_lane_recorded
    else:
        return top_3_BM_recorded, top_3_BM_dict_cars_on_bridge, top_3_BM_one_lane_recorded


def save_to_SQL():
    return

def load_max_BM_from_excel() -> tuple[pd.DataFrame]:
    xls = pd.ExcelFile("Final_Year_Project/max_BM_record.xlsx")
    df1 = pd.read_excel(xls, 'Max BM Value', header=1)
    
    top_3_BM_recorded = float(df1['top_3_BM_recorded'])
    day = int(df1['day'])

    top_3_BM_df_cars_on_bridge = pd.read_excel(xls, 'Max BM Traffic Configuration', header=1)
    return day, top_3_BM_recorded, top_3_BM_df_cars_on_bridge


def string_to_list(string: str)-> list[float]| int:

    if string == '0':
        return 0
    elif type(string) != str:
        return string
    else: 
        tmp = string.lstrip('[" ')
        tmp = tmp.rstrip(']" ')
        if ' ' in tmp:
            tmp.split(' ')
            while True: 
                tmp1 = tmp.replace('  ',' ')
                if tmp1==tmp: 
                    break 
                else:
                    tmp = tmp1
            return [float(x) for x in tmp.split(' ')]
        else: 
            return [float(tmp)]


def load_sim_info_from_csv(filename: str):
    output = pd.read_csv(filename, header=None)
    output = output.to_numpy()
    return output

def load_days_from_csv(filename: str):
    output = pd.read_csv(filename, header=None).iloc[0,0]
    return output

def load_output_from_csv(filename: str):
    output = pd.read_csv(filename, header=None)


    for column in range(len(output.columns)):
        if column == 1:
            continue
        else: 
            output.iloc[:,column] = output.iloc[:,column].astype(float)
    
    return output.to_numpy()


def save_to_csv(filename:str , item: np.float64 | np.int8 | list[float])-> None:
    if type(item) == int:
        with open(filename, 'w') as csv_file:
            csv.writer(csv_file).writerow([item])
    else:
        pd.DataFrame(item).to_csv(filename,header=False, index=False) 
    return

def load_top_BM_from_csv(filepath: str):
    top_3_BM_recorded = np.genfromtxt(filepath, delimiter=',')
    return top_3_BM_recorded

def load_top_BM_np_from_csv(filepath1: str, filepath2: str, filepath3: str):
    top_3_BM_dict_cars_on_bridge = {}
    
    for key in range(3):
        filepath = [filepath1,filepath2, filepath3][key]

        
        top_3_BM_dict_cars_on_bridge[key] = load_output_from_csv(filepath)

    return top_3_BM_dict_cars_on_bridge

def check_and_update_sim_info(day:int, simulation_information: np.float64| np.int8, top_3_BM_recorded: np.float64):
    year = day / 365
    if year in simulation_information[0]:
        simulation_information[1][np.where(simulation_information[0] == year)] = np.max(top_3_BM_recorded)
    return simulation_information

def Monte_Carlo_Simulation_One_Year():
    hours_in_a_day = 1
    total_lane = 3
    EV_proportion = 1
    traffic_growth = 3.75
    debugger_mode = False
    multiprocessing = False

    top_3_BM_recorded_filepath = 'Simulation_Results/top_3_BM.csv'
    top_3_BM_one_lane_recorded_filepath = 'Simulation_Results/top_3_BM_one_lane.csv'
    day_filepath = 'Simulation_Results/day.csv'
    simulation_information_filepath = 'Simulation_Results/simulation_information.csv'
    top_3_BM_dict_cars_on_bridge0_filepath = 'Simulation_Results/max_BM_record1.csv'
    top_3_BM_dict_cars_on_bridge1_filepath = 'Simulation_Results/max_BM_record2.csv'
    top_3_BM_dict_cars_on_bridge2_filepath = 'Simulation_Results/max_BM_record3.csv'

    new_BM_recorded = np.array([0,0,0])
    new_BM_dict_cars_on_bridge = {}
    new_BM_one_lane_recorded = np.array([0,0,0])



    top_3_BM_recorded = np.array([0,0,0])
    top_3_BM_dict_cars_on_bridge = {}

    top_3_BM_one_lane_recorded = np.array([0,0,0])
    
    if multiprocessing:
        total_number_of_days = 365
    else:
        total_number_of_days = 365 * 2400

    simulation_information = load_sim_info_from_csv(simulation_information_filepath)
    day = int(load_days_from_csv(day_filepath))


    while day <= total_number_of_days:
         

        if debugger_mode:
            np_generated_daily_traffic = load_output_from_csv('Simulation_Results/generated_daily_vehicle_stream.csv')
        else:
            np_generated_daily_traffic = generate_daily_vehicle_stream(hours_in_a_day, total_lane, EV_proportion, traffic_growth)
            top_3_BM_recorded, top_3_BM_dict_cars_on_bridge, top_3_BM_one_lane_recorded =  load_top_BM_from_csv(top_3_BM_recorded_filepath), load_top_BM_np_from_csv(top_3_BM_dict_cars_on_bridge0_filepath,top_3_BM_dict_cars_on_bridge1_filepath,top_3_BM_dict_cars_on_bridge2_filepath), load_top_BM_from_csv(top_3_BM_one_lane_recorded_filepath)
        new_top_3_BM_recorded, new_top_3_BM_dict_cars_on_bridge, new_top_3_BM_one_lane_recorded = load_assessment(np_generated_daily_traffic, top_3_BM_recorded, top_3_BM_dict_cars_on_bridge, hours_in_a_day, total_lane, top_3_BM_one_lane_recorded, new_BM_recorded, new_BM_dict_cars_on_bridge, new_BM_one_lane_recorded, multiprocessing)
        day += 1
    return top_3_BM_recorded, top_3_BM_dict_cars_on_bridge, top_3_BM_one_lane_recorded

def main():
    while True:
        Monte_Carlo_Simulation_One_Year()
    return


